In [1]:
import freesasa as fr
import Bio.PDB as bp
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

Calcualte Surface Area of pdb

In [182]:
#selecting files, need to strip hydrogens for freesasa?
#make a list of names of the pdb files and run them through in a loop?
input_dir = '/Users/junyi/Desktop/research/Amyloid_Alter/'
input_dir_loop = '/Users/junyi/Desktop/research/amyloids_row/amyloids_pdb/'

input_dir = input_dir_loop

output_dir = '/Users/junyi/Desktop/research/'
pdb_name = 'pdb5w7v_nH'
pdb_file = input_dir_loop + pdb_name + '.pdb'
path_to_database = '/Users/junyi/Desktop/research/database/vdMs'

# deltaSA = SA(complex) - SA(monomer1) - SA(monomer2)

In [3]:
#make a list of names of the pdb files and run them through in a loop?
my_file = open(input_dir+"amyloid_list.txt", "r")
content = my_file.read()
pdb_list = content.split("\n")
my_file.close()

for_loop = np.array(pdb_list)
pdb_list = []

for name in for_loop:
    name = "pdb"+name+"_nH"
    pdb_list = pdb_list + [name]

pdb_list = pdb_list[:-1]


In [4]:
#How many chains?
parser = bp.PDBParser()
structure = parser.get_structure(pdb_name, pdb_file)
chain_id = []
chains_structure = []

for chain in structure[0]:
        chain_id = chain_id + [chain.get_id()]
        chains_structure = chains_structure + [chain]

chain_id
chains_structure
#dictionary of tables
pdb_dic = {}


/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 971
  warnings.warn(


detaSA = SA(complex) - SA(monomer1) - SA(monomer2)

Remeber to change the Chain Name/Code when calculating surface area. Doesn't always follow A and B

In [5]:
#another version, I am confused too
model_result, sasa_classes = fr.calcBioPDB(structure[0])
print("Total Area:", model_result.totalArea(), "A2")

for chain in chains_structure:
    result, sasa_classes = fr.calcBioPDB(chain)
    print("Chain", chain.get_id() + ":", result.totalArea(), "A2")

Total Area: 10657.524880350446 A2
Chain 1: 1636.5757594240142 A2
Chain 3: 1559.6510192101953 A2
Chain 2: 1531.406758016579 A2
Chain 6: 1682.0843751172424 A2
Chain 5: 1544.795707216807 A2
Chain 4: 1501.1114552458932 A2
Chain 7: 1639.789373501226 A2
Chain 0: 934.6340296564302 A2
Chain 8: 822.5621185317885 A2


In [6]:
#changes the residueArea object to residue area
def to_num(obj):
    if obj == 0:
        return 0
    return obj.total

def to_name(obj):
    if obj == 0:
        return 0
    return obj.residueType

residue_area = pd.DataFrame.from_dict(model_result.residueAreas()).fillna(0).applymap(to_num)
residue_names = pd.DataFrame.from_dict(model_result.residueAreas()).fillna(0).applymap(to_name).iloc[:,0]
cluster_table = residue_area.join(residue_names,rsuffix=pdb_name).rename(columns={residue_area.columns[0]+pdb_name:"res_name"})
#for clustering I think I can go through a lot of pdb and manually find the solvent exposed chains and use it to 
# train data to classify into solvent exposed or not (uses regression or decision tree)
#can also just treat data as unlabelled and cluster like in this case

#classify instead

In [7]:
#somehow find the two strands that have the most exposure to the solvent and take them out
to_change = residue_area.copy()
to_change.apply(stats.zscore).apply(np.abs)
to_change.loc['Total',:] = to_change.sum(axis=0)
z_score_totals = to_change.loc['Total'].to_frame()
non_exposed = z_score_totals[(np.abs(stats.zscore(z_score_totals)) < 1)].index.to_numpy()
#the chains that are not exposed to the solvent
non_exposed
old_ver = non_exposed
old_ver

array(['1', '3', '2', '6', '5', '4'], dtype=object)

In [8]:
old_ver = old_ver.astype('int')
new = []
for i in old_ver:
    new = new + [chr(ord('@')+i)]
non_exposed = np.array(new)
a = dict(zip(old_ver.astype('str'),non_exposed))
residue_area.rename(columns=a,inplace=True)

In [9]:
#do we want this with all of the chains or just the ones without exposure?
print("Total Residue Sums:", sum(residue_area.sum()))

Total Residue Sums: 10657.524880350446


In [10]:
solvent_exposed_pdb = []
for pdb in pdb_list:
    pdb_name = pdb
    pdb_file = input_dir_loop + pdb_name + '.pdb'
    #parse pdb
    parser = bp.PDBParser()
    structure = parser.get_structure(pdb_name, pdb_file)
    chain_id = []
    chains_structure = []
    for chain in structure[0]:
        chain_id = chain_id + [chain.get_id()]
        chains_structure = chains_structure + [chain]
    #the models and chains calculations
    model_result, sasa_classes = fr.calcBioPDB(structure[0])
    #changes the residueArea object to residue area
    residue_area = pd.DataFrame.from_dict(model_result.residueAreas()).fillna(0).applymap(to_num)
    #somehow find the two strands that have the most exposure to the solvent and take them out
    non_exposed = residue_area.copy()
    if len(residue_area.columns) > 2:
        to_change = residue_area.copy()
        to_change.apply(stats.zscore).apply(np.abs)
        to_change.loc['Total',:] = to_change.sum(axis=0)
        z_score_totals = to_change.loc['Total'].to_frame()

        #kmeanModel = KMeans(n_clusters=2).fit(non_exposed.T)
        #aaa = non_exposed.T
        #aaa['clustering'] = kmeanModel.labels_
        #non_exposed = aaa[aaa['clustering']==1].index.to_numpy()
        #doesn't work because the clustering results come out randomly
        non_exposed = z_score_totals[(np.abs(stats.zscore(z_score_totals)) < 1)].index.to_numpy()
        if non_exposed[0].isnumeric():
            print(pdb_name)
            old_ver = non_exposed.copy()
            non_exposed = non_exposed.astype('int')
            new = []
            for i in non_exposed:
                new = new + [chr(ord('@')+i)]
            non_exposed = np.array(new)
            a = dict(zip(old_ver.astype('str'),non_exposed))
            residue_area.rename(columns=a,inplace=True)
        non_exposed = np.char.capitalize(non_exposed.astype('str'))
    #changes the residueArea object to residue names
    residue_area_names = pd.DataFrame.from_dict(model_result.residueAreas()).fillna(0).applymap(to_name).iloc[:,0]
    residue_area_named = residue_area.join(residue_area_names, rsuffix=pdb_name)
    residue_area_named = residue_area_named.rename(columns={residue_area_named.columns[-1]:"res_name"})
    non_exposed_named = residue_area_named
    if len(residue_area.columns) > 2:
        non_exposed_named = residue_area_named.loc[:,non_exposed.tolist()]
    non_exposed_named['avg'] = non_exposed_named.mean(axis=1)
    #ideal final lookup table would have a index that means nothing, columns of the residue number, residue name,
    # #  residue average sasa, and original pdb file name
    if len(residue_area.columns) <= 2:
        non_exposed_named.reset_index(inplace=True)
        solvent_exposed_pdb = solvent_exposed_pdb + [pdb_name]
    if len(residue_area.columns) > 2:
        non_exposed_named = non_exposed_named.join(residue_area_named.iloc[:,-1:]).reset_index()
    non_exposed_named = non_exposed_named.rename(columns = {'index':'res_num'})
    #maybe add a column with the pdb file name repeated for however many times there are rows in the table? 
    # for the final, just append all the tables together by row
    #store the individual tables in a dictionary?
    #chain id should be on its own
    non_exposed_named.insert(0, 'PDB', pdb_name)
    non_exposed_named = non_exposed_named.loc[:,~non_exposed_named.columns.duplicated()]
    pdb_dic[pdb_name] = non_exposed_named

/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 9391
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 15348
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 29104
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 29138
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 43510
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' 

pdb6W0O_nH


/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 14710
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 15161
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain R is discontinuous at line 2848.
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'U' for Atom (name=UNK) with given element 'X'
  warnings.warn(msg, PDBConstructionWarning)
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2849.
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConst

pdb6XFM_nH


/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 39581
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 61337
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2711
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 4409
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3693
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuo

pdb5w7v_nH


/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3130
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 4661
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 4929
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 5068
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3849
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous

In [11]:
#find the total columns
#columns in dataframe = total - 4
columns_width = []
for i in pdb_dic.values():
    columns_width = columns_width + [len(i.columns)]
a = np.array(columns_width)
a = a - 4
max(a)
column_names = ['PDB','res_num','A','B','C','D','E','F','G','H',
'I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','avg','res_name']
new_df = pd.DataFrame(columns=column_names)

In [12]:
no_expose_chains = new_df.copy()
for pdb_df in pdb_dic.keys():
    no_expose_chains = no_expose_chains.append(pdb_dic[pdb_df], ignore_index=True)
no_expose_chains

,PDB,res_num,A,B,C,D,E,F,G,H,...,S,T,U,V,W,X,Y,Z,avg,res_name
0,pdb2beg_nH,17,NaN,97.794887,96.242199,65.447717,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.494934,LEU
1,pdb2beg_nH,18,NaN,37.943971,39.140082,47.900973,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.661675,VAL
2,pdb2beg_nH,19,NaN,12.896385,11.454364,21.458401,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.269717,PHE
3,pdb2beg_nH,20,NaN,42.262372,57.714567,48.854846,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.610595,PHE
4,pdb2beg_nH,21,NaN,10.963000,8.146369,11.769877,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.293082,ALA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11827,pdb7ob4_nH,119,NaN,NaN,0.093305,0.134970,0.289039,0.479125,0.510506,0.126971,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.283738,THR
11828,pdb7ob4_nH,120,NaN,NaN,22.807875,25.904715,25.747539,26.368077,26.048419,24.872782,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.106648,ALA
11829,pdb7ob4_nH,121,NaN,NaN,0.000000,0.066480,0.118352,0.226947,0.292808,0.14018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.132216,VAL
11830,pdb7ob4_nH,122,NaN,NaN,52.116955,56.384040,55.245134,57.230288,56.877539,55.496831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.056749,VAL


In [13]:
pattern = '|'.join(solvent_exposed_pdb)
no_exposed_df = no_expose_chains.loc[~no_expose_chains['PDB'].str.contains(pattern, case=False)]
solvent_exposed_df = no_expose_chains.loc[no_expose_chains['PDB'].str.contains(pattern, case=False)]

Different story from above

In [201]:
pdb_name = 'pdb2mxu_nH'
pdb_file = input_dir + pdb_name + '.pdb'
path_to_database = '/Users/junyi/Desktop/research/database/vdMs'

#How many chains?
parser = bp.PDBParser()
structure = parser.get_structure(pdb_name, pdb_file)
residue_dic = {}
lst_residues = []
#only one chain is needed

model_result, sasa_classes = fr.calcBioPDB(structure[0])

for chain in chains_structure:
    result, sasa_classes = fr.calcBioPDB(chain)

for chain in structure[0]:
        chain_id = chain.get_id()
        for residue in chain:
                lst_residues = lst_residues + [residue.get_resname()]
        break

#make recurrsive function to make a list
#input is a list or array of residue names
#output is a list of list of 5 residue with keys of counter
#counter is the number of residues in the sublist
#index is where to start
lst = []
def sub_five(input,counter,index,lst,dictionary):
        res = input[index]
        lst = lst + [res]
        if index + 1 >= len(input):
                return
        if counter == 4:
                index = index - 3
                counter = 0
                dictionary[index] = lst
                lst = []
                return sub_five(input,counter,index,lst,dictionary)
        counter = counter + 1
        index = index + 1
        return sub_five(input,counter,index,lst,dictionary)

sub_five(lst_residues,0,0,lst,residue_dic)
fragment_sasa_table = pd.DataFrame.from_dict(residue_dic)
fragment_sasa_table

/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 28833
  warnings.warn(


{}


,1,2,3,4,5,6,7,8,9,10,...,18,19,20,21,22,23,24,25,26,27
0,GLU,VAL,HIS,HIS,GLN,LYS,LEU,VAL,PHE,PHE,...,LYS,GLY,ALA,ILE,ILE,GLY,LEU,MET,VAL,GLY
1,VAL,HIS,HIS,GLN,LYS,LEU,VAL,PHE,PHE,ALA,...,GLY,ALA,ILE,ILE,GLY,LEU,MET,VAL,GLY,GLY
2,HIS,HIS,GLN,LYS,LEU,VAL,PHE,PHE,ALA,GLU,...,ALA,ILE,ILE,GLY,LEU,MET,VAL,GLY,GLY,VAL
3,HIS,GLN,LYS,LEU,VAL,PHE,PHE,ALA,GLU,ASP,...,ILE,ILE,GLY,LEU,MET,VAL,GLY,GLY,VAL,VAL
4,GLN,LYS,LEU,VAL,PHE,PHE,ALA,GLU,ASP,VAL,...,ILE,GLY,LEU,MET,VAL,GLY,GLY,VAL,VAL,ILE


In [131]:
residue_area = pd.DataFrame.from_dict(model_result.residueAreas()).fillna(0).applymap(to_num)
residue_names = pd.DataFrame.from_dict(model_result.residueAreas()).fillna(0).applymap(to_name).iloc[:,0]
cluster_table = residue_area.join(residue_names,rsuffix=pdb_name).rename(columns={residue_area.columns[0]+pdb_name:"res_name"})

In [15]:
#find the index of end of the fragment
def find_cont(names,fragment):
    counter = 0
    for ind,res in names.items():
        if res == fragment[counter]:
            counter = counter + 1
            if counter == 5:
                return ind
            continue
        counter = 0
    return "None"

In [124]:
#import the manual cutoff table

np.random.seed(99)

cuttoff_table = pd.read_csv('/Users/junyi/Desktop/research/Amyloid_Alter/residue_cutoff.csv').iloc[:,:-3]
cutoff = cuttoff_table.interpolate().fillna(method='backfill')

In [123]:
dummy_analysis = no_exposed_df.iloc[:,-2:].iloc[3918,:]
dummy_analysis[1]

# linear data analysis on the table
    # linear by the means of (residue sasa - minimum cutoff) / (max cutoff - min cutoff)

# dummy code: let residue be dummy_analysis

#write a function where we get the sasa score and the residue name of the input 
# object and we return its score according to the table

def analyzed_score(series):
    sasa = series[0]
    residue = series[1]
    #find it in table
    compiled = cutoff[cutoff['Amino Acid'] == residue].iloc[:,1:].T.iloc[:,0]
    iterr = np.split(compiled, len(compiled)/2)
    #count cutoff in pairs
    cutoff_score = []
    for i in iterr:
        range = i[1] - i[0]
        sasa_min = sasa - i[0]
        if sasa < i[0]:
            cutoff_score = cutoff_score + [0]
        elif sasa > i[1]:
            cutoff_score = cutoff_score + [1]
        else:
            cutoff_score = cutoff_score + [sasa_min/range]
    return np.average(cutoff_score)

analyzed_score(dummy_analysis)

0.5599241451800256

In [220]:
#cutoff approach calculations, use no_expose_chains directly
no_expose_chains.head()
simplified = no_expose_chains[['PDB','avg','res_name']]
# do pdb2beg_nH for example
specific = simplified[simplified['PDB']=='pdb2mxu_nH'].copy()
cutoff_scores = []
for i in np.arange(0,len(specific.iloc[:,1:])):
    series = specific.iloc[:,-2:].iloc[i,:]
    cutoff_scores = cutoff_scores + [analyzed_score(series)]

specific['cutoff_scores'] = cutoff_scores

lst_cutoffs = np.array(specific['cutoff_scores']).tolist()
c_lst = []
cutoffs_dic = {}

sub_five(lst_cutoffs,0,0,c_lst,cutoffs_dic)
fragment_cutoff = pd.DataFrame.from_dict(cutoffs_dic)
fragment_cutoff = fragment_cutoff.T
fragment_cutoff['avg'] = fragment_cutoff.sum(axis=1)

fragment_with_cutoff = fragment_sasa_table.T
fragment_with_cutoff['cutoff_sum'] = fragment_cutoff['avg']
fragment_with_cutoff

,0,1,2,3,4,cutoff_sum
1,GLU,VAL,HIS,HIS,GLN,4.842127
2,VAL,HIS,HIS,GLN,LYS,4.842127
3,HIS,HIS,GLN,LYS,LEU,4.704568
4,HIS,GLN,LYS,LEU,VAL,4.704568
5,GLN,LYS,LEU,VAL,PHE,3.704568
6,LYS,LEU,VAL,PHE,PHE,2.862442
7,LEU,VAL,PHE,PHE,ALA,2.729325
8,VAL,PHE,PHE,ALA,GLU,2.866884
9,PHE,PHE,ALA,GLU,ASP,2.866884
10,PHE,ALA,GLU,ASP,VAL,3.025318


In [173]:
#sasa average calcualtion by compiling all pdb files

rep = no_exposed_df.iloc[:,-1]
res_sum = []
unidentified = []
for name,data in fragment_sasa_table.iteritems():
    contents = data.values
    end = find_cont(rep,contents)
    if end == "None":
        exp_rep = no_exposed_df.iloc[:,-1]
        end = find_cont(exp_rep,contents)
        if end == "None":
            unidentified = unidentified + [name]
            res_sum = res_sum + [end]
            continue
    start = end - 4
    res_sum = res_sum + [no_exposed_df.loc[start:end,:]['avg'].sum()]

fragment_sasa_sum = fragment_sasa_table.T
fragment_sasa_sum['sum'] = res_sum
fragment_sasa_sum

,0,1,2,3,4,sum
1,LYS,THR,LYS,GLU,GLY,223.833479
2,THR,LYS,GLU,GLY,VAL,220.495815
3,LYS,GLU,GLY,VAL,VAL,636.300702
4,GLU,GLY,VAL,VAL,HIS,109.405922
5,GLY,VAL,VAL,HIS,GLY,79.782796
6,VAL,VAL,HIS,GLY,VAL,116.638263
7,VAL,HIS,GLY,VAL,ALA,None
8,HIS,GLY,VAL,ALA,THR,165.236814
9,GLY,VAL,ALA,THR,VAL,146.905314
10,VAL,ALA,THR,VAL,ALA,189.569897


In [17]:
#clustering
from sklearn.cluster import KMeans
from scipy.spatial import distance
distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,10) 
X = fragment_sasa_table

kmeanModel = KMeans(n_clusters=2, random_state=0).fit(X)
aaa = X
aaa['clustering'] = kmeanModel.labels_
aaa

ValueError: could not convert string to float: 'LYS'

In [ ]:
#no exclusive version
no_ex_pdb = {}
for pdb in pdb_list:
    pdb_name = pdb
    pdb_file = input_dir_loop + pdb_name + '.pdb'
    #parse pdb
    parser = bp.PDBParser()
    structure = parser.get_structure(pdb_name, pdb_file)
    chain_id = []
    chains_structure = []
    for chain in structure[0]:
        chain_id = chain_id + [chain.get_id()]
        chains_structure = chains_structure + [chain]
    #the models and chains calculations
    model_result, sasa_classes = fr.calcBioPDB(structure[0])
    #changes the residueArea object to residue area
    residue_area = pd.DataFrame.from_dict(model_result.residueAreas()).fillna(0).applymap(to_num)
    #somehow find the two strands that have the most exposure to the solvent and take them out
    residue_area_names = pd.DataFrame.from_dict(model_result.residueAreas()).fillna(0).applymap(to_name).iloc[:,0]
    residue_area_named = residue_area.join(residue_area_names, rsuffix=pdb_name)
    residue_area_named = residue_area_named.rename(columns={residue_area_named.columns[-1]:"res_name"})
    residue_area_named['avg'] = residue_area_named.mean(axis=1)
    residue_area_named = residue_area_named.rename(columns = {'index':'res_num'})
    #maybe add a column with the pdb file name repeated for however many times there are rows in the table? 
    # for the final, just append all the tables together by row
    #store the individual tables in a dictionary?
    #chain id should be on its own
    residue_area_named.insert(0, 'PDB', pdb_name)
    residue_area_named = residue_area_named.loc[:,~residue_area_named.columns.duplicated()]
    no_ex_pdb[pdb_name] = residue_area_named

no_ex_df = new_df.copy()
for pdb_df in no_ex_pdb.keys():
    no_ex_df = no_ex_df.append(no_ex_pdb[pdb_df], ignore_index=True)

/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 9391
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 15348
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 29104
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 29138
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 43510
  warnings.warn(
/Users/junyi/opt/miniconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' 

In [ ]:
#classifying, this is a try of the first pdb I found
chains_dic = {}
chain_series = no_ex_df[no_ex_df['PDB'] == 'pdb2beg_nH']['A']
sub_five(chain_series,0,0,lst,chains_dic)
chain_series_table = pd.DataFrame.from_dict(chains_dic,orient='index')
chain_series_table['sum'] = chain_series_table.T.sum()
chain_series_table

#continue making the tables for the non-exposed chain and then find the exposed partts in the pdb file to look up in the table
#if forget what is exposed, ask hailey to send picture of pdb

NameError: name 'no_ex_df' is not defined

In [ ]:
#classifying, chain values
no_ex_df[no_ex_df['PDB'] == 'pdb2beg_nH']

NameError: name 'no_ex_df' is not defined

In [ ]:
#when have time Tuesday, go through a lot of pdb files and 
# look for solvent exposed chains, then find their sasa value through tables (the no exposed and solvent exposed)
#also go through the 5 residue method and get a lot of data
#do the same for the none exposed chains too
#make decision tree accordingly